In [59]:
from lectures_to_viewers import getViewersToLectures
from video_annotations import ml_004 as vid
from get_seek_chains import getSeekChainsFast

In [60]:
import json

In [ ]:
all_viewers = set()
viewed_at_least_half = set()
viewed_at_least_quarter = set()
num_lecture_starts = 0

for viewer,lectures_viewed in getViewersToLectures().iteritems():
  all_viewers.add(viewer)
  num_lecture_starts += len(lectures_viewed)
  if len(lectures_viewed) >= 56:
    viewed_at_least_half.add(viewer)
  if len(lectures_viewed) >= 28:
    viewed_at_least_quarter.add(viewer)
    
print num_lecture_starts
print len(all_viewers)
print len(viewed_at_least_half)
print len(viewed_at_least_quarter)
print len(vid.video_lengths.keys())

1377238
59641
8935
16430
113


In [ ]:

#user_to_num_lectures_started = {}
all_viewers_v2 = set()
for lecture_id,user_to_start_events in json.load(open('video_to_user_to_startzero_events_orig.json')).iteritems():
  for user,start_events in user_to_start_events.iteritems():
    all_viewers_v2.add(user)
    #if user not in user_to_num_lectures_started:
    #  user_to_num_lectures_started[user] = 0
    #user_to_num_lectures_started[user] += 1

print len(all_viewers_v2)

In [ ]:
seek_events_total = 0
seek_chains_total = 0

for lecture_id in vid.video_lengths.keys():
  seek_chains = getSeekChainsFast(lecture_id)
  for seek_chain in seek_chains:
    seek_events_total += len(seek_chain.seek_events) 
    seek_chains_total += 1

print seek_events_total
print seek_chains_total

In [ ]:
print sum([len(x) for x in vid.in_video_quiz_times.values()])
print sum([len(x) for x in vid.slide_transitions.values()])

In [ ]:
#video_to_user_to_end_events = json.load(open('video_to_user_to_end_events.json'))

#for lecture_id,user_to_end_events in video_to_user_to_end_events.iteritems():
#  

In [ ]:
# of in-video quizzes answered total
import csv
session_id_to_eventing_id = {}
session_id_to_coursera_id = {}
coursera_id_to_eventing_id = {}
coursera_id_to_public_id = {}
session_id_to_public_id = {}

for line in csv.DictReader(open('hash_mapping.csv')):
  coursera_id_to_eventing_id[line['user_id']] = line['eventing_user_id']
  coursera_id_to_public_id[line['user_id']] = line['public_user_id']

for line in csv.DictReader(open('id_map.csv')):
  #session_id_to_eventing_id[line['session_user_id']] = line['eventing_user_id']
  user_id = line['coursera_user_id']
  session_id = line['session_user_id']
  session_id_to_coursera_id[session_id] = user_id
  if user_id not in coursera_id_to_eventing_id:
    continue
  public_id = coursera_id_to_public_id[user_id]
  eventing_id = coursera_id_to_eventing_id[user_id]
  session_id_to_eventing_id[session_id] = eventing_id
  session_id_to_public_id[session_id] = public_id

In [ ]:
print len(session_id_to_coursera_id)
print len(session_id_to_eventing_id)

In [ ]:
print len(session_id_to_eventing_id)
print session_id_to_eventing_id['c693d0b484da0667b2bbbf157a2cd6c95ffe6376']

In [ ]:
import csv
import json
from memoized import memoized

@memoized
def loadjson(table_name):
  return json.load(open('/lfs/local/0/geza/ml-004_data_export/' + table_name + '.json'))

def get_table_path(table_name):
  csv_path = '/lfs/local/0/geza/ml-004_data_export/'
  return csv_path + table_name + '.csv'

def get_table_data(table_name):
  return csv.DictReader(open(get_table_path(table_name)), doublequote=False, delimiter=',', escapechar='\\')

def print_first_lines(table_name, n=10):
  for x in get_table_data(table_name):
    if n <= 0:
      break
    n -= 1
    print x

In [ ]:

invideo_quiz_itemids = {}
review_quiz_itemids = {}

for line in csv.DictReader(open('quiz_metadata.csv')):
  if line['deleted'] != '0':
    continue
  if line['quiz_type'] == 'video':
    if int(line['id']) > 144:
      continue
    invideo_quiz_itemids[line['id']] = True
  if line['quiz_type'] == 'quiz':
    if int(line['id']) > 141:
      continue
    review_quiz_itemids[line['id']] = True

#print invideo_quiz_itemids

In [ ]:
'''
invideo_quiz_itemids = set()

for line in get_table_data('quiz_metadata'):
  if line['deleted'] != '0':
    continue
  if line['duration'] != '0':
    continue
  quiz_id = int(line['id'])
  if quiz_id > 144:
    continue
  if line['quiz_type'] != 'video':
    continue
  invideo_quiz_itemids.add(quiz_id)
'''

In [ ]:
#print len(invideo_quiz_itemids)

In [ ]:
quiz_xml = loadjson('kvs_course.quiz.xml')
for key,value in quiz_xml:
  try:
    tree = et.fromstring(value.encode('utf-8'))
    print list(tree.iter('title'))[0].text
  except:
    continue

In [ ]:

'''
import xml.etree.ElementTree as et
#import lxml.etree as et

nonzero_quizzes = []

invideo_quiz_itemids = {}
#review_quiz_itemids = {}


quiz_xml = loadjson('kvs_course.quiz.xml')
for key,value in quiz_xml:
  try:
    parts = key.split('.')
    quiz_id = parts[1].split(':')[1]
    tree = et.fromstring(value.encode('utf-8'))
    score = list(tree.iter('maximum_score'))[0].text
    score = int(score)
    title = list(tree.iter('title'))[0].text
    #print value
    question_groups = [x.text for x in list(tree.iter('question_group'))]
    print title, question_groups
    #if ' min)' in title: #and score == 1:
    #  nonzero_quizzes.append([title, quiz_id])
    #  invideo_quiz_itemids[quiz_id] = title
  except:
    continue

'''
#print nonzero_quizzes

In [ ]:
print len(invideo_quiz_itemids)
#print len(set(invideo_quiz_itemids.values()))
#for x in nonzero_quizzes:
#  print x

In [ ]:
print review_quiz_itemids

In [ ]:

'''
import xml.etree.ElementTree as et
#import lxml.etree as et

review_questions = []

review_quiz_itemids = {}

quiz_xml = json.load(open('/lfs/local/0/geza/ml-004_data_export/kvs_course.quiz.xml.json'))
for key,value in quiz_xml:
  try:
    parts = key.split('.')
    quiz_id = parts[1].split(':')[1]
    tree = et.fromstring(value.encode('utf-8'))
    score = list(tree.iter('maximum_score'))[0].text
    score = int(score)
    title = list(tree.iter('title'))[0].text
    retry_delay = list(tree.iter('retry_delay'))[0].text
    retry_delay = int(retry_delay)
    if ' min)' not in title and score > 1 and retry_delay == 600:
      review_questions.append([title, quiz_id])
      review_quiz_itemids[quiz_id] = title
  except:
    continue

#print nonzero_quizzes
'''

In [ ]:
#print len(review_questions)
#for x in review_questions:
#  print x
#print len(set(review_questions))
print len(review_quiz_itemids)
print len(set(review_quiz_itemids.values()))

In [ ]:
print len(invideo_quiz_itemids.keys())

In [ ]:
print len(all_viewers)

In [ ]:
# from phpserialize import unserialize

In [ ]:

'''
import xml.etree.ElementTree as et
#import lxml.etree as et

review_quiz_itemids = {}

quiz_xml = loadjson('kvs_course.quiz.xml')
for key,value in quiz_xml:
  try:
    #print value
    parts = key.split('.')
    quiz_id = parts[1].split(':')[1]
    tree = et.fromstring(value.encode('utf-8'))
    score = list(tree.iter('maximum_score'))[0].text
    score = int(score)
    title = list(tree.iter('title'))[0].text
    retry_delay = list(tree.iter('retry_delay'))[0].text
    retry_delay = int(retry_delay)
    if 'Quiz' in title:
      print title, quiz_id
    #if ' min)' not in title and score > 1 and retry_delay == 600:
    #  review_questions.append([title, quiz_id])
    #  review_quiz_itemids[quiz_id] = title
  except:
    continue
'''

#print nonzero_quizzes


In [ ]:
print review_quiz_itemids

In [ ]:
quiz_counts = {}

for line in loadjson('kvs_course.quiz.saved'):
  key,value = line
  if key.startswith('saved.quiz_id:'):
    parts = key.split('.')
    quiz_id = parts[1].split(':')[1]
    user_id = parts[2].split(':')[1]
    if quiz_id not in review_quiz_itemids:
      continue
    #if int(quiz_id) >= 145:
    #  continue
    if quiz_id not in quiz_counts:
      quiz_counts[quiz_id] = 0
    #if value['answers'] == {}:
    #  continue
    quiz_counts[quiz_id] += 1

print json.dumps(quiz_counts, sort_keys=True, indent=2, separators=(',', ': '))

In [ ]:
quiz_counts = {}

for line in loadjson('kvs_course.quiz.saved'):
  key,value = line
  if key.startswith('saved.quiz_id:'):
    parts = key.split('.')
    quiz_id = parts[1].split(':')[1]
    user_id = parts[2].split(':')[1]
    if int(quiz_id) >= 145:
      continue
    if quiz_id not in quiz_counts:
      quiz_counts[quiz_id] = 0
    if value['answers'] == {}:
      continue
    quiz_counts[quiz_id] += 1

print json.dumps(quiz_counts, sort_keys=True, indent=2, separators=(',', ': '))

In [ ]:
non_invideo_quiz_items = 0
unknown_session_id = 0
unknown_eventing_id = 0

user_to_invideo_quiz_answers = {}
user_to_review_quiz_answers = {}

users_before_feb = {}

for line in csv.DictReader(open('quiz_submission_metadata.csv')):
  item_id = line['item_id']
  #print line['submission_time']
  #if int(line['submission_time']) > 1388538061: # newer than jan 1, 2014
  if int(line['submission_time']) > 1391185339: # newer than jan 31, 2014
    continue
  if item_id not in invideo_quiz_itemids and item_id not in review_quiz_itemids:
    non_invideo_quiz_items += 1
    continue
  session_id = line['session_user_id']
  users_before_feb[session_id] = True
  if session_id not in session_id_to_eventing_id:
    unknown_session_id += 1
    continue
  eventing_id = session_id_to_eventing_id[session_id]
  coursera_id = session_id_to_coursera_id[session_id]
  public_id = session_id_to_public_id[session_id]
  # in video quizzes
  if item_id in invideo_quiz_itemids:
    if session_id not in user_to_invideo_quiz_answers:
      user_to_invideo_quiz_answers[session_id] = {}
    if item_id not in user_to_invideo_quiz_answers:
      user_to_invideo_quiz_answers[session_id][item_id] = []
    user_to_invideo_quiz_answers[session_id][item_id].append(line)
  # review quizzes
  if item_id in review_quiz_itemids:
    if session_id not in user_to_review_quiz_answers:
      user_to_review_quiz_answers[session_id] = {}
    if item_id not in user_to_review_quiz_answers:
      user_to_review_quiz_answers[session_id][item_id] = []
    user_to_review_quiz_answers[session_id][item_id].append(line)
  #eventing_id = coursera_id

In [ ]:

'''
non_invideo_quiz_items = 0
unknown_session_id = 0
unknown_eventing_id = 0

user_to_invideo_quiz_answers = {}
user_to_review_quiz_answers = {}

users_before_feb = {}

for line in loadjson('kvs_course.quiz.saved'):
  key,value = line
  if key.startswith('saved.quiz_id:'):
    parts = key.split('.')
    quiz_id = parts[1].split(':')[1]
    user_id = parts[2].split(':')[1]
    item_id = quiz_id
    session_id = user_id
    if user_id not in all_viewers:
      continue
    parsed_value = value
    submission_time = parsed_value['saved_time']
    if int(submission_time) > 1391185339: # newer than jan 31, 2014
      continue
    if item_id not in invideo_quiz_itemids and item_id not in review_quiz_itemids:
      non_invideo_quiz_items += 1
      continue
    # in video quizzes
    if item_id in invideo_quiz_itemids:
      if parsed_value['answers'] == {}:
        continue
      if session_id not in user_to_invideo_quiz_answers:
        user_to_invideo_quiz_answers[session_id] = {}
      if item_id not in user_to_invideo_quiz_answers:
        user_to_invideo_quiz_answers[session_id][item_id] = []
      user_to_invideo_quiz_answers[session_id][item_id].append(line)
    # review quizzes
    if item_id in review_quiz_itemids:
      if session_id not in user_to_review_quiz_answers:
        user_to_review_quiz_answers[session_id] = {}
      if item_id not in user_to_review_quiz_answers:
        user_to_review_quiz_answers[session_id][item_id] = []
      user_to_review_quiz_answers[session_id][item_id].append(line)
'''


In [ ]:
'''
non_invideo_quiz_items = 0
unknown_session_id = 0
unknown_eventing_id = 0

user_to_invideo_quiz_answers = {}
user_to_review_quiz_answers = {}

users_before_feb = {}

for line in get_table_data('kvs_course.quiz'):
  key = line['key']
  value = line['value']
  if key.startswith('saved.quiz_id:'):
    parts = key.split('.')
    quiz_id = parts[1].split(':')[1]
    user_id = parts[2].split(':')[1]
    item_id = quiz_id
    session_id = user_id
    if user_id not in all_viewers:
      continue
    parsed_value = unserialize(unserialize(value))
    submission_time = parsed_value['saved_time']
    if int(submission_time) > 1391185339: # newer than jan 31, 2014
      continue
    if item_id not in invideo_quiz_itemids and item_id not in review_quiz_itemids:
      non_invideo_quiz_items += 1
      continue
    # in video quizzes
    if item_id in invideo_quiz_itemids:
      if session_id not in user_to_invideo_quiz_answers:
        user_to_invideo_quiz_answers[session_id] = {}
      if item_id not in user_to_invideo_quiz_answers:
        user_to_invideo_quiz_answers[session_id][item_id] = []
      user_to_invideo_quiz_answers[session_id][item_id].append(line)
    # review quizzes
    if item_id in review_quiz_itemids:
      if session_id not in user_to_review_quiz_answers:
        user_to_review_quiz_answers[session_id] = {}
      if item_id not in user_to_review_quiz_answers:
        user_to_review_quiz_answers[session_id][item_id] = []
      user_to_review_quiz_answers[session_id][item_id].append(line)
'''

In [ ]:
"""
non_invideo_quiz_items = 0
unknown_session_id = 0
unknown_eventing_id = 0

user_to_invideo_quiz_answers = {}
user_to_review_quiz_answers = {}

users_before_feb = {}

for line in csv.DictReader(open('quiz_submission_metadata.csv')):
  item_id = line['item_id']
  #print line['submission_time']
  #if int(line['submission_time']) > 1388538061: # newer than jan 1, 2014
  if int(line['submission_time']) > 1391185339: # newer than jan 31, 2014
    continue
  if item_id not in invideo_quiz_itemids and item_id not in review_quiz_itemids:
    non_invideo_quiz_items += 1
    continue
  session_id = line['session_user_id']
  users_before_feb[session_id] = True
  if session_id not in session_id_to_eventing_id:
    unknown_session_id += 1
    continue
  eventing_id = session_id_to_eventing_id[session_id]
  coursera_id = session_id_to_coursera_id[session_id]
  public_id = session_id_to_public_id[session_id]
  # in video quizzes
  if item_id in invideo_quiz_itemids:
    if session_id not in user_to_invideo_quiz_answers:
      user_to_invideo_quiz_answers[session_id] = {}
    if item_id not in user_to_invideo_quiz_answers:
      user_to_invideo_quiz_answers[session_id][item_id] = []
    user_to_invideo_quiz_answers[session_id][item_id].append(line)
  # review quizzes
  if item_id in review_quiz_itemids:
    if session_id not in user_to_review_quiz_answers:
      user_to_review_quiz_answers[session_id] = {}
    if item_id not in user_to_review_quiz_answers:
      user_to_review_quiz_answers[session_id][item_id] = []
    user_to_review_quiz_answers[session_id][item_id].append(line)
  #eventing_id = coursera_id
"""
  

In [ ]:
print len(users_before_feb)

In [ ]:
print non_invideo_quiz_items
print unknown_session_id
print unknown_eventing_id
print len(user_to_invideo_quiz_answers)
print len(user_to_review_quiz_answers)

In [ ]:
print non_invideo_quiz_items
print unknown_session_id
print unknown_eventing_id
print len(user_to_invideo_quiz_answers)
print len(user_to_review_quiz_answers)

In [ ]:
num_invideo_quiz_submissions = 0
num_invideo_quizzes_submitted = 0

for user,invideo_quiz_answers in user_to_invideo_quiz_answers.iteritems():
  num_invideo_quiz_submissions += len(invideo_quiz_answers)
  if len(invideo_quiz_answers) > 0:
    num_invideo_quizzes_submitted += 1

print num_invideo_quiz_submissions
print num_invideo_quizzes_submitted

In [ ]:
num_review_quiz_submissions = 0
num_review_quizzes_submitted = 0

for user,review_quiz_answers in user_to_review_quiz_answers.iteritems():
  num_review_quiz_submissions += len(review_quiz_answers)
  if len(review_quiz_answers) > 0:
    num_review_quizzes_submitted += 1

print num_review_quiz_submissions
print num_review_quizzes_submitted

In [ ]:
#print sum([len(x) for x in vid['in_video_quiz_times'].values()])

In [ ]:
#import gzip

#max_timestamp = 0
#for line in gzip.open('/lfs/local/0/geza/datastage.stanford.edu/csdl_01/ml-004_clickstream.gz'):
#  data = json.loads(line)
#  if 'timestamp' in data:
#    max_timestamp = max(data['timestamp'], max_timestamp)


In [ ]:
#print max_timestamp

In [ ]:
num_lectures_started_to_num_users = [0]*114
num_lectures_finished_to_num_users = [0]*114

user_to_num_lectures_started = {}
for user,lectures_viewed in getViewersToLectures().iteritems():
  if len(lectures_viewed) == 0:
    continue
  user_to_num_lectures_started[user] = len(lectures_viewed)

'''
user_to_num_lectures_started = {}
for lecture_id,user_to_start_events in json.load(open('video_to_user_to_startzero_events_orig.json')).iteritems():
  for user,start_events in user_to_start_events.iteritems():
    if user not in user_to_num_lectures_started:
      user_to_num_lectures_started[user] = 0
    user_to_num_lectures_started[user] += 1
'''

for user,num_lectures_started in user_to_num_lectures_started.items():
  num_lectures_started_to_num_users[num_lectures_started] += 1

user_to_num_lectures_finished = {}
for lecture_id,user_to_end_events in json.load(open('video_to_user_to_end_events.json')).iteritems():
  for user,end_events in user_to_end_events.iteritems():
    if user not in user_to_num_lectures_started:
      continue
    if user not in user_to_num_lectures_finished:
      user_to_num_lectures_finished[user] = 0
    user_to_num_lectures_finished[user] += 1

for user,num_lectures_finished in user_to_num_lectures_finished.items():
  num_lectures_finished_to_num_users[num_lectures_finished] += 1



In [ ]:
print sum(user_to_num_lectures_finished.values())

In [ ]:
#print num_lectures_started_to_num_users

In [ ]:
print sum(num_lectures_started_to_num_users)

In [ ]:
# missing:
# 1, 8, 12-20, 33-37, 39-40, 52, 65
# additional which aren't in the quiz_metadata list at all: [73,74,75] + [66, 67, 68, 69, 70, 71]

# questions for which answers are missing

num_printed = 0
for user,invideo_quiz_answers in user_to_invideo_quiz_answers.iteritems():
  if len(invideo_quiz_answers) == 100:
    answered_answers = sorted([int(x) for x in invideo_quiz_answers])
    answered_answers_set = set(answered_answers)
    missing_questions = []
    for x in range(2, 130):
      if x not in answered_answers_set:
        if x in ([73,74,75] + [66, 67, 68, 69, 70, 71]):
          continue
        missing_questions.append(x)
    print missing_questions
    num_printed += 1
    if num_printed >= 10:
      break


In [ ]:
users_who_earned_certificate = set()

for line in get_table_data('course_grades'):
  user = line['session_user_id']
  if user not in user_to_num_lectures_started:
    continue
  grade = float(line['normal_grade'])
  if grade >= 80.0:
    users_who_earned_certificate.add(user)

In [ ]:
grades_to_num_users = [0]*101

user_to_grade = {}
for line in get_table_data('course_grades'):
  user = line['session_user_id']
  if user not in user_to_num_lectures_started:
    continue
  grade = float(line['normal_grade'])
  user_to_grade[user] = grade

for user,grade in user_to_grade.items():
  grade = int(round(grade))
  if grade < 0:
    grade = 0
  if grade > 100:
    grade = 100
  grades_to_num_users[grade] += 1

In [ ]:
at_least_n_grade = [sum(grades_to_num_users[i:]) for i in range(len(grades_to_num_users))]

In [ ]:
print at_least_n_grade

In [ ]:
user_to_lectures_viewed = {}

for line in get_table_data('lecture_submission_metadata'):
  action = line['action']
  user = line['session_user_id']
  lecture = line['item_id']
  submission_time = int(line['submission_time'])
  if submission_time > 1391185339: # newer than jan 31, 2014
    continue
  if action == 'view':
    if user not in user_to_lectures_viewed:
      user_to_lectures_viewed[user] = set()
    user_to_lectures_viewed[user].add(lecture)

num_lecture_views_to_num_users = [0]*114

for user,lectures_viewed in user_to_lectures_viewed.items():
  num_lecture_views = len(lectures_viewed)
  if num_lecture_views >= 113:
    num_lecture_views = 113
  num_lecture_views_to_num_users[num_lecture_views] += 1

at_least_n_views = [sum(num_lecture_views_to_num_users[i:]) for i in range(len(num_lecture_views_to_num_users))]

In [ ]:
print sum([len(x) for x in user_to_lectures_viewed.values()])

In [ ]:
print at_least_n_views

In [ ]:
user_to_lectures_downloaded = {}

for line in get_table_data('lecture_submission_metadata'):
  action = line['action']
  user = line['session_user_id']
  lecture = line['item_id']
  submission_time = int(line['submission_time'])
  if submission_time > 1391185339: # newer than jan 31, 2014
    continue
  if action == 'download':
    if user not in user_to_lectures_downloaded:
      user_to_lectures_downloaded[user] = set()
    user_to_lectures_downloaded[user].add(lecture)

num_lecture_downloads_to_num_users = [0]*114

for user,lectures_downloaded in user_to_lectures_downloaded.items():
  num_lecture_downloads = len(lectures_downloaded)
  if num_lecture_downloads >= 113:
    num_lecture_downloads = 113
  num_lecture_downloads_to_num_users[num_lecture_downloads] += 1

at_least_n_downloads = [sum(num_lecture_downloads_to_num_users[i:]) for i in range(len(num_lecture_downloads_to_num_users))]

In [ ]:
print sum([len(x) for x in user_to_lectures_downloaded.values()])

In [ ]:
print at_least_n_downloads

In [ ]:
num_quizzes_answered_to_num_users = [0]*110
num_review_quizzes_answered_to_num_users = [0]*19

user_to_num_invideo_quizzes = {}
for user,invideo_quiz_answers in user_to_invideo_quiz_answers.iteritems():
  #user_to_num_invideo_quizzes[user] = len(invideo_quiz_answers)
  item_ids = invideo_quiz_answers.keys() #set([x['item_id'] for x in invideo_quiz_answers])
  user_to_num_invideo_quizzes[user] = len(item_ids)

for user,num_invideo_quizzes in user_to_num_invideo_quizzes.items():
  if num_invideo_quizzes > 109:
    num_invideo_quizzes = 109
  num_quizzes_answered_to_num_users[num_invideo_quizzes] += 1

user_to_num_review_quizzes = {}
for user,review_quiz_answers in user_to_review_quiz_answers.iteritems():
  #user_to_num_invideo_quizzes[user] = len(invideo_quiz_answers)
  item_ids = review_quiz_answers.keys() #set([x['item_id'] for x in review_quiz_answers])
  #print len(item_ids)
  user_to_num_review_quizzes[user] = len(item_ids)

for user,num_review_quizzes in user_to_num_review_quizzes.items():
  if num_review_quizzes > 18:
    num_review_quizzes = 18
  num_review_quizzes_answered_to_num_users[num_review_quizzes] += 1

In [ ]:
#print user_to_num_invideo_quizzes

In [ ]:
print num_lectures_started_to_num_users
print num_lectures_finished_to_num_users
print num_quizzes_answered_to_num_users
print num_review_quizzes_answered_to_num_users

In [ ]:
at_least_n_lectures_watched = [sum(num_lectures_started_to_num_users[i:]) for i in range(len(num_lectures_started_to_num_users))]
at_least_n_lectures_finished = [sum(num_lectures_finished_to_num_users[i:]) for i in range(len(num_lectures_finished_to_num_users))]
at_least_n_invideo_quizzes = [sum(num_quizzes_answered_to_num_users[i:]) for i in range(len(num_quizzes_answered_to_num_users))]
at_least_n_review_quizzes = [sum(num_review_quizzes_answered_to_num_users[i:]) for i in range(len(num_review_quizzes_answered_to_num_users))]

In [ ]:
from bokeh.plotting import *
output_notebook()


In [ ]:
print at_least_n_lectures_finished[1]

In [ ]:
print at_least_n_views[1]

In [ ]:
p1 = figure(
  title='User engagement with lectures and in-video quizzes on ML4',
  y_axis_label='number of users',
  x_axis_label='completed at least x% of material',
  plot_width=800,
  plot_height=400,
)
p1.line(
  [100.0*float(i+1)/113 for i in range(113)], # x coordinates
  [at_least_n_views[i+1] for i in range(113)], # y coordinates
  color='#ff0000',
  legend='began watching lecture',
)
p1.line(
  [100.0*float(i+1)/113 for i in range(113)], # x coordinates
  [at_least_n_lectures_finished[i+1] for i in range(113)], # y coordinates
  color='#00ff00',
  legend='finished watching lecture',  
)
p1.line(
  [100.0*float(i+1)/100 for i in range(100)], # x coordinates
  [at_least_n_invideo_quizzes[i+1] for i in range(100)], # y coordinates
  color='#0000ff',
  legend='submitted in-video quiz',
)
#p1.line(
#  [100.0*float(1+i)/113 for i in range(113)], # x coordinates
#  [at_least_n_downloads[i+1] for i in range(113)], # y coordinates
#  color='#115555',
#  legend='downloaded lecture'
#)
p1.line(
  [100.0*float(1+i)/100 for i in range(100)], # x coordinates
  [at_least_n_grade[i+1] for i in range(100)], # y coordinates
  color='#000000',
  legend='grade (from assignments and exams)'
)


#p1.line(
#  [100.0*float(1+i)/18 for i in range(18)], # x coordinates
#  [at_least_n_review_quizzes[i+1] for i in range(18)], # y coordinates
#  color='#ffff00',
#)

show(p1)

In [ ]:
p1 = figure(
  title='User engagement with lectures and in-video quizzes on ML4',
  y_axis_label='number of users',
  x_axis_label='completed at least x% of material',
  plot_width=800,
  plot_height=400,
)
p1.line(
  [100.0*float(i+1)/113 for i in range(113)], # x coordinates
  [at_least_n_views[i+1] for i in range(113)], # y coordinates
  color='#ff0000',
  legend='began watching lecture',
)
p1.line(
  [100.0*float(i+1)/113 for i in range(113)], # x coordinates
  [at_least_n_lectures_finished[i+1] for i in range(113)], # y coordinates
  color='#00ff00',
  legend='finished watching lecture',  
)
p1.line(
  [100.0*float(i+1)/100 for i in range(100)], # x coordinates
  [at_least_n_invideo_quizzes[i+1] for i in range(100)], # y coordinates
  color='#0000ff',
  legend='submitted in-video quiz',
)
p1.line(
  [100.0*float(1+i)/113 for i in range(113)], # x coordinates
  [at_least_n_downloads[i+1] for i in range(113)], # y coordinates
  color='#115555',
  legend='downloaded lecture'
)
p1.line(
  [100.0*float(1+i)/100 for i in range(100)], # x coordinates
  [at_least_n_grade[i+1] for i in range(100)], # y coordinates
  color='#000000',
  legend='grade (from assignments and exams)'
)


#p1.line(
#  [100.0*float(1+i)/18 for i in range(18)], # x coordinates
#  [at_least_n_review_quizzes[i+1] for i in range(18)], # y coordinates
#  color='#ffff00',
#)

show(p1)

In [ ]:
p1 = figure(
  title='User engagement with lectures and in-video quizzes on ML4',
  y_axis_label='number of users',
  x_axis_label='completed at least x% of material',
  plot_width=800,
  plot_height=400,
)
p1.line(
  [100.0*float(1+i)/113 for i in range(113)], # x coordinates
  [at_least_n_lectures_watched[i+1] for i in range(113)], # y coordinates
  color='#ff0000',
  legend='began watching lecture',
)
p1.line(
  [100.0*float(1+i)/113 for i in range(113)], # x coordinates
  [at_least_n_lectures_finished[i+1] for i in range(113)], # y coordinates
  color='#00ff00',
  legend='finished watching lecture',  
)
p1.line(
  [100.0*float(1+i)/100 for i in range(109)], # x coordinates
  [at_least_n_invideo_quizzes[i+1] for i in range(100)], # y coordinates
  color='#0000ff',
  legend='submitted in-video quiz',
)
p1.line(
  [100.0*float(1+i)/18 for i in range(18)], # x coordinates
  [at_least_n_review_quizzes[i+1] for i in range(18)], # y coordinates
  color='#000000',
  legend='review question',
)

show(p1)

In [335]:
users_who_opened_course = set() # session ids of users who opened course
for session_id in getViewersToLectures().keys():
  #if eventing_id not in eventing_to_session_id:
  #  continue
  #session_id = eventing_to_session_id[eventing_id]
  users_who_opened_course.add(session_id)

In [336]:
num_lectures_started_to_num_users = [0]*114
num_lectures_finished_to_num_users = [0]*114

user_to_num_lectures_started = {}
for user,lectures_viewed in getViewersToLectures().iteritems():
  if user not in users_who_earned_certificate:
    continue
  #if len(lectures_viewed) == 0:
  #  continue
  user_to_num_lectures_started[user] = len(lectures_viewed)

'''
user_to_num_lectures_started = {}
for lecture_id,user_to_start_events in json.load(open('video_to_user_to_startzero_events_orig.json')).iteritems():
  for user,start_events in user_to_start_events.iteritems():
    if user not in user_to_num_lectures_started:
      user_to_num_lectures_started[user] = 0
    user_to_num_lectures_started[user] += 1
'''

for user,num_lectures_started in user_to_num_lectures_started.items():
  num_lectures_started_to_num_users[num_lectures_started] += 1

user_to_num_lectures_finished = {}
for lecture_id,user_to_end_events in json.load(open('video_to_user_to_end_events.json')).iteritems():
  for user,end_events in user_to_end_events.iteritems():
    if user not in users_who_earned_certificate:
      continue
    if user not in user_to_num_lectures_started:
      continue
    if user not in user_to_num_lectures_finished:
      user_to_num_lectures_finished[user] = 0
    user_to_num_lectures_finished[user] += 1

for user,num_lectures_finished in user_to_num_lectures_finished.items():
  num_lectures_finished_to_num_users[num_lectures_finished] += 1



In [337]:
num_quizzes_answered_to_num_users = [0]*110
num_review_quizzes_answered_to_num_users = [0]*19

user_to_num_invideo_quizzes = {}
for user,invideo_quiz_answers in user_to_invideo_quiz_answers.iteritems():
  #user_to_num_invideo_quizzes[user] = len(invideo_quiz_answers)
  if user not in users_who_earned_certificate:
    continue
  item_ids = invideo_quiz_answers.keys() #set([x['item_id'] for x in invideo_quiz_answers])
  user_to_num_invideo_quizzes[user] = len(item_ids)

for user,num_invideo_quizzes in user_to_num_invideo_quizzes.items():
  if num_invideo_quizzes > 109:
    num_invideo_quizzes = 109
  num_quizzes_answered_to_num_users[num_invideo_quizzes] += 1

user_to_num_review_quizzes = {}
for user,review_quiz_answers in user_to_review_quiz_answers.iteritems():
  if user not in users_who_earned_certificate:
    continue
  #user_to_num_invideo_quizzes[user] = len(invideo_quiz_answers)
  item_ids = review_quiz_answers.keys() #set([x['item_id'] for x in review_quiz_answers])
  #print len(item_ids)
  user_to_num_review_quizzes[user] = len(item_ids)

for user,num_review_quizzes in user_to_num_review_quizzes.items():
  if num_review_quizzes > 18:
    num_review_quizzes = 18
  num_review_quizzes_answered_to_num_users[num_review_quizzes] += 1

In [338]:
at_least_n_lectures_watched = [sum(num_lectures_started_to_num_users[i:]) for i in range(len(num_lectures_started_to_num_users))]
at_least_n_lectures_finished = [sum(num_lectures_finished_to_num_users[i:]) for i in range(len(num_lectures_finished_to_num_users))]
at_least_n_invideo_quizzes = [sum(num_quizzes_answered_to_num_users[i:]) for i in range(len(num_quizzes_answered_to_num_users))]
at_least_n_review_quizzes = [sum(num_review_quizzes_answered_to_num_users[i:]) for i in range(len(num_review_quizzes_answered_to_num_users))]

In [339]:
grades_to_num_users = [0]*101

user_to_grade = {}
for line in get_table_data('course_grades'):
  user = line['session_user_id']
  if user not in user_to_num_lectures_started:
    continue
  if user not in users_who_earned_certificate:
    continue
  grade = float(line['normal_grade'])
  user_to_grade[user] = grade

for user,grade in user_to_grade.items():
  grade = int(round(grade))
  if grade < 0:
    grade = 0
  if grade > 100:
    grade = 100
  grades_to_num_users[grade] += 1

In [340]:
at_least_n_grade = [sum(grades_to_num_users[i:]) for i in range(len(grades_to_num_users))]

In [341]:
#print at_least_n_grade

In [342]:
user_to_lectures_downloaded = {}

for line in get_table_data('lecture_submission_metadata'):
  action = line['action']
  user = line['session_user_id']
  lecture = line['item_id']
  if user not in users_who_earned_certificate:
    continue
  submission_time = int(line['submission_time'])
  if submission_time > 1391185339: # newer than jan 31, 2014
    continue
  if action == 'download':
    if user not in user_to_lectures_downloaded:
      user_to_lectures_downloaded[user] = set()
    user_to_lectures_downloaded[user].add(lecture)

num_lecture_downloads_to_num_users = [0]*114

for user,lectures_downloaded in user_to_lectures_downloaded.items():
  num_lecture_downloads = len(lectures_downloaded)
  if num_lecture_downloads >= 113:
    num_lecture_downloads = 113
  num_lecture_downloads_to_num_users[num_lecture_downloads] += 1

at_least_n_downloads = [sum(num_lecture_downloads_to_num_users[i:]) for i in range(len(num_lecture_downloads_to_num_users))]

In [343]:
user_to_lectures_viewed = {}

for line in get_table_data('lecture_submission_metadata'):
  action = line['action']
  user = line['session_user_id']
  lecture = line['item_id']
  submission_time = int(line['submission_time'])
  if submission_time > 1391185339: # newer than jan 31, 2014
    continue
  if user not in users_who_earned_certificate:
    continue
  if action == 'view':
    if user not in user_to_lectures_viewed:
      user_to_lectures_viewed[user] = set()
    user_to_lectures_viewed[user].add(lecture)

num_lecture_views_to_num_users = [0]*114

for user,lectures_viewed in user_to_lectures_viewed.items():
  num_lecture_views = len(lectures_viewed)
  if num_lecture_views >= 113:
    num_lecture_views = 113
  num_lecture_views_to_num_users[num_lecture_views] += 1

at_least_n_views = [sum(num_lecture_views_to_num_users[i:]) for i in range(len(num_lecture_views_to_num_users))]

In [354]:
p1 = figure(
  title='User engagement among users who earned certificate',
  y_axis_label='number of users',
  x_axis_label='completed at least x% of material',
  plot_width=800,
  plot_height=400,
)
p1.line(
  [100.0*float(i+1)/112 for i in range(112)], # x coordinates
  [at_least_n_views[i+1] for i in range(112)], # y coordinates
  color='#ff0000',
  legend='began watching lecture',
)
p1.line(
  [100.0*float(i+1)/112 for i in range(112)], # x coordinates
  [at_least_n_lectures_finished[i+1] for i in range(112)], # y coordinates
  color='#00ff00',
  legend='finished watching lecture',  
)
p1.line(
  [100.0*float(i+1)/100 for i in range(100)], # x coordinates
  [at_least_n_invideo_quizzes[i+1] for i in range(100)], # y coordinates
  color='#0000ff',
  legend='submitted in-video quiz',
)
#p1.line(
#  [100.0*float(1+i)/113 for i in range(113)], # x coordinates
#  [at_least_n_downloads[i+1] for i in range(113)], # y coordinates
#  color='#115555',
#  legend='downloaded lecture'
#)
p1.line(
  [100.0*float(1+i)/100 for i in range(100)], # x coordinates
  [at_least_n_grade[i+1] for i in range(100)], # y coordinates
  color='#000000',
  legend='grade (from assignments and exams)'
)
#p1.line(
#  [100.0*float(1+i)/18 for i in range(18)], # x coordinates
#  [at_least_n_review_quizzes[i+1] for i in range(18)], # y coordinates
#  color='#ffff00',
#)

p1.legend.orientation = "bottom_left"

'''
p1.legend.legend_spacing = 0
p1.legend.legend_padding = 1
#p1.legend.label_height = 3
p1.legend.label_text_font_size = '9pt'
#p1.legend.label_text_font = 'times'
p1.legend.border_line_width = 0
#p1.legend.glyph_height = 10
p1.legend.border_line_alpha = 0
'''

show(p1)

In [344]:
p1 = figure(
  title='User engagement among users who earned certificate',
  y_axis_label='number of users',
  x_axis_label='completed at least x% of material',
  plot_width=800,
  plot_height=400,
)
p1.line(
  [100.0*float(i+1)/112 for i in range(112)], # x coordinates
  [at_least_n_views[i+1] for i in range(112)], # y coordinates
  color='#ff0000',
  legend='began watching lecture',
)
p1.line(
  [100.0*float(i+1)/112 for i in range(112)], # x coordinates
  [at_least_n_lectures_finished[i+1] for i in range(112)], # y coordinates
  color='#00ff00',
  legend='finished watching lecture',  
)
p1.line(
  [100.0*float(i+1)/100 for i in range(100)], # x coordinates
  [at_least_n_invideo_quizzes[i+1] for i in range(100)], # y coordinates
  color='#0000ff',
  legend='submitted in-video quiz',
)
p1.line(
  [100.0*float(1+i)/113 for i in range(113)], # x coordinates
  [at_least_n_downloads[i+1] for i in range(113)], # y coordinates
  color='#115555',
  legend='downloaded lecture'
)
p1.line(
  [100.0*float(1+i)/100 for i in range(100)], # x coordinates
  [at_least_n_grade[i+1] for i in range(100)], # y coordinates
  color='#000000',
  legend='grade (from assignments and exams)'
)
#p1.line(
#  [100.0*float(1+i)/18 for i in range(18)], # x coordinates
#  [at_least_n_review_quizzes[i+1] for i in range(18)], # y coordinates
#  color='#ffff00',
#)

p1.legend.orientation = "bottom_left"
p1.legend.legend_spacing = 0
p1.legend.legend_padding = 1
#p1.legend.label_height = 3
p1.legend.label_text_font_size = '9pt'
#p1.legend.label_text_font = 'times'
p1.legend.border_line_width = 0
#p1.legend.glyph_height = 10
p1.legend.border_line_alpha = 0

show(p1)

In [345]:
import json

In [346]:
num_start_events = 0
num_videos_started = 0

for lecture_id,user_to_start_events in json.load(open('video_to_user_to_startzero_events_orig.json')).iteritems():
  for user,start_events in user_to_start_events.iteritems():
    num_start_events += len(start_events)
    if len(start_events) > 0:
      num_videos_started += 1

print num_start_events
print num_videos_started

1962563
1260676


In [347]:
num_end_events = 0
num_videos_ended = 0

for lecture_id,user_to_end_events in json.load(open('video_to_user_to_end_events.json')).iteritems():
  for user,end_events in user_to_end_events.iteritems():
    num_end_events += len(end_events)
    if len(end_events) > 0:
      num_videos_ended += 1

print num_end_events
print num_videos_ended

1055775
976933


In [348]:
# at least num lectures watched to num users = sum([0:i])

In [349]:
users_who_started_at_least_one_lecture = set()

for lecture_id,user_to_start_events in json.load(open('video_to_user_to_startzero_events_orig.json')).iteritems():
  for user,start_events in user_to_start_events.iteritems():
    if user not in users_who_started_at_least_one_lecture:
      users_who_started_at_least_one_lecture.add(user)

print len(users_who_started_at_least_one_lecture)

48131


In [350]:
user_to_lectures_finished = {}

for lecture_id,user_to_end_events in json.load(open('video_to_user_to_startzero_events.json')).iteritems():
  for user,end_events in user_to_end_events.iteritems():
    if user not in user_to_lectures_finished:
      user_to_lectures_finished[user] = []
    user_to_lectures_finished[user].append(lecture_id)

num_users_who_finished_one_lecture = 0
num_users_who_finished_quarter_lectures = 0
num_users_who_finished_half_lectures = 0

for user,lectures_finished in user_to_lectures_finished.iteritems():
  num_lectures_finished = len(lectures_finished)
  if num_lectures_finished >= 1:
    num_users_who_finished_one_lecture += 1
  if num_lectures_finished >= 28:
    num_users_who_finished_quarter_lectures += 1
  if num_lectures_finished >= 56:
    num_users_who_finished_half_lectures += 1

print num_users_who_finished_one_lecture
print num_users_who_finished_quarter_lectures
print num_users_who_finished_half_lectures

48131
15652
8232


In [351]:
user_to_lectures_finished = {}

for lecture_id,user_to_end_events in json.load(open('video_to_user_to_end_events.json')).iteritems():
  for user,end_events in user_to_end_events.iteritems():
    if user not in user_to_lectures_finished:
      user_to_lectures_finished[user] = []
    user_to_lectures_finished[user].append(lecture_id)

num_users_who_finished_one_lecture = 0
num_users_who_finished_quarter_lectures = 0
num_users_who_finished_half_lectures = 0

for user,lectures_finished in user_to_lectures_finished.iteritems():
  num_lectures_finished = len(lectures_finished)
  if num_lectures_finished >= 1:
    num_users_who_finished_one_lecture += 1
  if num_lectures_finished >= 28:
    num_users_who_finished_quarter_lectures += 1
  if num_lectures_finished >= 56:
    num_users_who_finished_half_lectures += 1

print num_users_who_finished_one_lecture
print num_users_who_finished_quarter_lectures
print num_users_who_finished_half_lectures

41643
11820
6127


In [352]:
# overall course level engagements - graphs showing # users vs course percentage completion



In [353]:
num_quiz_pauses = 0
num_quizzes_paused = 0

for lecture_id,user_to_quiz_pause_events in json.load(open('video_to_user_to_quiz_pause_events.json')).iteritems():
  for user,quizzes_pause_events in user_to_quiz_pause_events.iteritems():
    for quiz_idx,quiz_pause_events in enumerate(quizzes_pause_events):
      num_quiz_pauses += len(quiz_play_events)
      if len(quiz_play_events) > 0:
        num_quizzes_paused += 1

print num_quiz_pauses
print num_quizzes_paused

NameError: name 'quiz_play_events' is not defined

In [ ]:
num_quiz_answers = 0
num_quizzes_answered = 0

for lecture_id,user_to_quiz_play_events in json.load(open('video_to_user_to_quiz_play_events.json')).iteritems():
  for user,quizzes_play_events in user_to_quiz_play_events.iteritems():
    for quiz_idx,quiz_play_events in enumerate(quizzes_play_events):
      num_quiz_answers += len(quiz_play_events)
      if len(quiz_play_events) > 0:
        num_quizzes_answered += 1

print num_quiz_answers
print num_quizzes_answered

In [ ]:
import glob
print glob.glob('*.json')